1:16 
[Link](https://www.linkedin.com/learning/nlp-with-python-for-machine-learning-essential-training/implementation-removing-punctuation?u=74412268)

Steps to NLP for ML:

1) Raw text - mdoel cannot read text

2) Tokenizing - telling model what to read from file 

3) Clean text - remove stop words, punctuation, stemming, etc.

4) Vectorize - turn data to numeric form

5) Machine Learning - put into model for train/test
    - classification model

## 1) Preprocess Data

In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 100) #set layout table

my_data = pd.read_csv("Restaurant_Reviews.tsv", sep="\t", header=None)
my_data.columns = ["Review","Positive/Negative rating"]

my_data[-5:]

,Review,Positive/Negative rating
996,I think food should have flavor and texture and both were lacking.,0
997,Appetite instantly gone.,0
998,Overall I was not impressed and would not go back.,0
999,"The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.",0
1000,"Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing ou...",0


## 2) Remove punctuation

In [2]:
#packages

import re

In [3]:
# Define a function to remove punctuation
def remove_punctuation(text):
    # Use regular expression to remove punctuation and replace with a space
    text = re.sub(r'[^\w\s]', ' ', text)
    return text

In [4]:
# Apply the function to the 'Message' column and create a new column 'Cleaned_Message'
my_data['Cleaned_Review'] = my_data['Review'].apply(remove_punctuation)

# Save the cleaned data to a new TSV file
my_data.to_csv('my_datacleaned_file.tsv', sep='\t', index=False)

In [5]:
my_clean_data = pd.read_csv("my_datacleaned_file.tsv", sep="\t")

my_clean_data[-5:]

,Review,Positive/Negative rating,Cleaned_Review
996,I think food should have flavor and texture and both were lacking.,0,I think food should have flavor and texture and both were lacking
997,Appetite instantly gone.,0,Appetite instantly gone
998,Overall I was not impressed and would not go back.,0,Overall I was not impressed and would not go back
999,"The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.",0,The whole experience was underwhelming and I think we ll just go to Ninja Sushi next time
1000,"Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing ou...",0,Then as if I hadn t wasted enough of my life there they poured salt in the wound by drawing ou...


## 3) Tokenization

Chops up words in the cleaned_review column. Good for adding vocabulary to model's bank and preprocessing of smaller managable pieces.

In [6]:
#uses re

def data_tokenization(text):
    #for every word in review
    
    #split word
    tokens = re.split("\W+", text)
    return tokens
    #tokenized = original.tokenizeFunct column
my_data['Tokenized'] = my_data['Cleaned_Review'].apply(lambda x:data_tokenization(x.lower()))

my_data[-5:]

,Review,Positive/Negative rating,Cleaned_Review,Tokenized
996,I think food should have flavor and texture and both were lacking.,0,I think food should have flavor and texture and both were lacking,"[i, think, food, should, have, flavor, and, texture, and, both, were, lacking, ]"
997,Appetite instantly gone.,0,Appetite instantly gone,"[appetite, instantly, gone, ]"
998,Overall I was not impressed and would not go back.,0,Overall I was not impressed and would not go back,"[overall, i, was, not, impressed, and, would, not, go, back, ]"
999,"The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.",0,The whole experience was underwhelming and I think we ll just go to Ninja Sushi next time,"[the, whole, experience, was, underwhelming, and, i, think, we, ll, just, go, to, ninja, sushi, ..."
1000,"Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing ou...",0,Then as if I hadn t wasted enough of my life there they poured salt in the wound by drawing ou...,"[then, as, if, i, hadn, t, wasted, enough, of, my, life, there, they, poured, salt, in, the, wou..."


## Test case sensitivity NLP

Purpose of using .lower method is to turn all capital letters to lower case so it knows the words are the same thing. 

To us, SONIC is the same as sonic, but since Python is case senitive, it see the two words as different things.

In [7]:
"SONIC" == "sonic"

False

## 4) Remove stopwords

In [8]:
import nltk

stopword = nltk.corpus.stopwords.words("english")

In [9]:
def remove_stopwords(tokenized_list):
    my_text = {word for word in tokenized_list if word not in stopword}
    return my_text

my_data['Remove_stop'] = my_data['Tokenized'].apply(lambda x: remove_stopwords(x))

my_data[-5:]

,Review,Positive/Negative rating,Cleaned_Review,Tokenized,Remove_stop
996,I think food should have flavor and texture and both were lacking.,0,I think food should have flavor and texture and both were lacking,"[i, think, food, should, have, flavor, and, texture, and, both, were, lacking, ]","{, flavor, lacking, think, food, texture}"
997,Appetite instantly gone.,0,Appetite instantly gone,"[appetite, instantly, gone, ]","{, appetite, gone, instantly}"
998,Overall I was not impressed and would not go back.,0,Overall I was not impressed and would not go back,"[overall, i, was, not, impressed, and, would, not, go, back, ]","{, impressed, go, overall, back, would}"
999,"The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.",0,The whole experience was underwhelming and I think we ll just go to Ninja Sushi next time,"[the, whole, experience, was, underwhelming, and, i, think, we, ll, just, go, to, ninja, sushi, ...","{, whole, go, next, ninja, sushi, think, experience, time, underwhelming}"
1000,"Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing ou...",0,Then as if I hadn t wasted enough of my life there they poured salt in the wound by drawing ou...,"[then, as, if, i, hadn, t, wasted, enough, of, my, life, there, they, poured, salt, in, the, wou...","{wound, , bring, enough, check, wasted, drawing, took, poured, salt, time, life}"


## 5) Split into Train/Test

In [11]:
from sklearn.model_selection import train_test_split

#use punctuation removed text, cleaned review, and tokenized version to test if model can properly categorize each review

X_train,X_test,Y_train,Y_test=train_test_split(my_data[["Cleaned_Review","Tokenized","Remove_stop"]], my_data["Positive/Negative rating"], test_size=.2)